In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
from lightgbm import LGBMClassifier 
import numpy as np
import pandas as pd

In [0]:
train=pd.read_csv('/content/gdrive/My Drive/train.csv')
test=pd.read_csv('/content/gdrive/My Drive/test.csv')
train.head()


,id,type,fiberID,psfMag_u,psfMag_g,psfMag_r,psfMag_i,psfMag_z,fiberMag_u,fiberMag_g,fiberMag_r,fiberMag_i,fiberMag_z,petroMag_u,petroMag_g,petroMag_r,petroMag_i,petroMag_z,modelMag_u,modelMag_g,modelMag_r,modelMag_i,modelMag_z
0,0,QSO,601,23.198224,21.431953,21.314148,21.176553,21.171444,22.581309,21.644453,21.657571,21.387653,21.572827,22.504317,21.431636,21.478312,21.145409,20.422446,22.749241,21.465534,21.364187,21.020605,21.147340
1,1,QSO,788,21.431355,20.708104,20.678850,20.703420,20.473229,21.868797,21.029773,20.967054,20.937731,21.063646,21.360701,20.778968,20.889705,20.639812,20.646660,21.492955,20.758527,20.753925,20.693389,20.512314
2,2,QSO,427,17.851451,16.727898,16.679677,16.694640,16.641788,18.171890,17.033098,16.999682,17.095999,17.076449,17.867253,16.738784,16.688874,16.744210,16.808006,17.818063,16.697434,16.641249,16.660177,16.688928
3,3,QSO,864,20.789900,20.040371,19.926909,19.843840,19.463270,21.039030,20.317165,20.217898,20.073852,19.794505,20.433907,19.993727,19.985531,19.750917,19.455117,20.770711,20.001699,19.889798,19.758113,19.552855
4,4,STAR_RED_DWARF,612,26.454969,23.058767,21.471406,19.504961,18.389096,25.700632,23.629122,21.742750,19.861718,18.810375,25.859229,22.426929,21.673551,19.610012,18.376141,24.877052,23.147993,21.475342,19.487330,18.375655


In [0]:
from sklearn.preprocessing import RobustScaler
#Robust Scaler로 Scaling할 Column을 선정(id,type,fiberID를 제외)
train_for_scaling=train.drop(columns=['id','type','fiberID'],axis=1)
train_for_scaling.head()

,psfMag_u,psfMag_g,psfMag_r,psfMag_i,psfMag_z,fiberMag_u,fiberMag_g,fiberMag_r,fiberMag_i,fiberMag_z,petroMag_u,petroMag_g,petroMag_r,petroMag_i,petroMag_z,modelMag_u,modelMag_g,modelMag_r,modelMag_i,modelMag_z
0,23.198224,21.431953,21.314148,21.176553,21.171444,22.581309,21.644453,21.657571,21.387653,21.572827,22.504317,21.431636,21.478312,21.145409,20.422446,22.749241,21.465534,21.364187,21.020605,21.147340
1,21.431355,20.708104,20.678850,20.703420,20.473229,21.868797,21.029773,20.967054,20.937731,21.063646,21.360701,20.778968,20.889705,20.639812,20.646660,21.492955,20.758527,20.753925,20.693389,20.512314
2,17.851451,16.727898,16.679677,16.694640,16.641788,18.171890,17.033098,16.999682,17.095999,17.076449,17.867253,16.738784,16.688874,16.744210,16.808006,17.818063,16.697434,16.641249,16.660177,16.688928
3,20.789900,20.040371,19.926909,19.843840,19.463270,21.039030,20.317165,20.217898,20.073852,19.794505,20.433907,19.993727,19.985531,19.750917,19.455117,20.770711,20.001699,19.889798,19.758113,19.552855
4,26.454969,23.058767,21.471406,19.504961,18.389096,25.700632,23.629122,21.742750,19.861718,18.810375,25.859229,22.426929,21.673551,19.610012,18.376141,24.877052,23.147993,21.475342,19.487330,18.375655


In [0]:
#Robust Scaling에 적용
rbscaler=RobustScaler()
print(rbscaler.fit(train_for_scaling))
train_robustscaled = rbscaler.transform(train_for_scaling) #robustscaling 적용
train_robustscaled = pd.DataFrame(train_robustscaled,columns=train_for_scaling.columns)
train_robustscaled = pd.concat([train[['id','type', 'fiberID']], train_robustscaled], axis=1) 
train_robustscaled #robustscaling과 type,fiberID 합치기

RobustScaler(copy=True, quantile_range=(25.0, 75.0), with_centering=True,
             with_scaling=True)


,id,type,fiberID,psfMag_u,psfMag_g,psfMag_r,psfMag_i,psfMag_z,fiberMag_u,fiberMag_g,fiberMag_r,fiberMag_i,fiberMag_z,petroMag_u,petroMag_g,petroMag_r,petroMag_i,petroMag_z,modelMag_u,modelMag_g,modelMag_r,modelMag_i,modelMag_z
0,0,QSO,601,0.927836,0.623783,0.753702,0.916931,1.041252,0.639184,0.634486,0.805768,0.905712,1.132499,0.838327,0.638332,0.770905,0.825931,0.748549,0.859066,0.664478,0.744135,0.795289,0.978342
1,1,QSO,788,0.223298,0.328228,0.496221,0.713509,0.757221,0.341950,0.386929,0.531161,0.720391,0.931005,0.389795,0.412532,0.573234,0.655629,0.823214,0.398328,0.419522,0.539673,0.685895,0.774402
2,2,QSO,427,-1.204186,-1.296932,-1.124607,-1.010056,-0.801392,-1.200267,-1.222701,-1.046600,-0.861999,-0.646814,-0.980354,-0.985231,-0.837531,-0.656544,-0.455081,-0.949423,-0.987518,-0.838240,-0.662475,-0.453490
3,3,QSO,864,-0.032482,0.055586,0.191466,0.343934,0.346375,-0.004199,0.099931,0.233233,0.364564,0.428779,0.026301,0.140866,0.269585,0.356218,0.426423,0.133448,0.157305,0.250155,0.373217,0.466269
4,4,STAR_RED_DWARF,612,2.226460,1.288028,0.817437,0.198234,-0.090595,1.940455,1.433796,0.839643,0.277187,0.039338,2.154141,0.982669,0.836473,0.308756,0.067117,1.639432,1.247397,0.781377,0.282689,0.088207
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199986,199986,QSO,568,-0.095930,0.217909,0.241661,0.420093,0.548298,-0.041957,0.264872,0.270482,0.467358,0.593728,0.184161,0.328591,0.316947,0.474749,0.540180,0.138063,0.269970,0.291783,0.474791,0.565716
199987,199987,GALAXY,362,1.759416,1.185329,0.654667,0.491704,0.400205,2.348040,1.107414,0.459206,0.318433,0.295249,3.344960,0.921627,0.305237,0.098497,0.209382,2.389373,0.833175,0.212136,0.116437,0.137232
199988,199988,STAR_RED_DWARF,146,1.091962,0.815997,0.355138,-0.252104,-0.522744,1.364844,0.893648,0.429626,-0.155780,-0.391384,2.202441,0.880978,0.385142,-0.062662,-0.251776,1.502419,0.814175,0.427970,-0.066405,-0.247054
199989,199989,GALAXY,85,0.119882,0.268249,0.233959,0.322353,0.377780,0.273255,0.161383,0.095711,0.170928,0.261944,-0.582843,-0.526213,-0.564692,-0.509540,-0.332802,-0.475946,-0.534496,-0.570515,-0.484095,-0.363270


In [0]:
#X,y 변수를 나누기
X=train_robustscaled.drop(columns=['type'],axis=1)
y=train_robustscaled['type']

In [0]:
#LGBM Model 구축하기
from lightgbm import LGBMClassifier
lgbm=LGBMClassifier(boosting_type='gbdt', objective='binary', num_leaves=10,
                                learning_rate=0.1, n_estimators=2000, max_depth=15,
                                bagging_fraction=0.9, feature_fraction=0.9, reg_lambda=0.2)

In [0]:
#모델에 학습시키기
lgbm.fit(X,y)

LGBMClassifier(bagging_fraction=0.9, boosting_type='gbdt', class_weight=None,
               colsample_bytree=1.0, feature_fraction=0.9,
               importance_type='split', learning_rate=0.1, max_depth=15,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=2000, n_jobs=-1, num_leaves=10, objective='binary',
               random_state=None, reg_alpha=0.0, reg_lambda=0.2, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [0]:
#Test 자료도 FiberID를 제외하고 Scaling하기
test_for_scaling=test.drop(columns=['id','fiberID'],axis=1)
test_for_scaling.head()

,psfMag_u,psfMag_g,psfMag_r,psfMag_i,psfMag_z,fiberMag_u,fiberMag_g,fiberMag_r,fiberMag_i,fiberMag_z,petroMag_u,petroMag_g,petroMag_r,petroMag_i,petroMag_z,modelMag_u,modelMag_g,modelMag_r,modelMag_i,modelMag_z
0,23.817399,22.508963,20.981106,18.517316,17.076079,25.053890,23.167848,21.335901,18.835858,17.434384,22.246697,22.796239,21.195315,18.584486,17.154284,25.391534,22.499435,21.011918,18.499341,17.091474
1,22.806983,21.937111,20.335770,20.000512,19.527369,22.498565,22.186000,20.618879,20.301204,20.030078,21.729831,21.837511,20.196128,19.967204,19.683671,22.475338,21.853442,20.173169,19.796757,19.567372
2,21.024250,19.235669,18.304061,17.808608,17.380113,21.205546,19.439533,18.344433,17.909690,17.538164,20.722629,18.710223,17.611851,17.158519,16.843986,20.579314,18.653338,17.562108,17.120529,16.708748
3,20.503424,20.286261,20.197204,20.162419,20.059832,20.976132,20.611498,20.567262,20.479318,20.277472,20.329269,20.385262,20.129157,20.206574,20.212342,20.479879,20.280943,20.150499,20.206221,20.092909
4,24.244851,22.668237,21.239333,19.284777,18.235939,25.681860,22.935289,21.642456,19.624926,18.591749,22.308298,22.957496,21.285033,19.299120,18.307526,25.489360,22.857290,21.191862,19.237964,18.280368


In [0]:
print(rbscaler.fit(test_for_scaling))
test_robustscaled = rbscaler.transform(test_for_scaling) #robustscaling 적용
test_robustscaled = pd.DataFrame(test_robustscaled,columns=test_for_scaling.columns)
test_robustscaled = pd.concat([train[['id','fiberID']], test_robustscaled], axis=1) 
test_robustscaled.head()

RobustScaler(copy=True, quantile_range=(25.0, 75.0), with_centering=True,
             with_scaling=True)


,id,fiberID,psfMag_u,psfMag_g,psfMag_r,psfMag_i,psfMag_z,fiberMag_u,fiberMag_g,fiberMag_r,fiberMag_i,fiberMag_z,petroMag_u,petroMag_g,petroMag_r,petroMag_i,petroMag_z,modelMag_u,modelMag_g,modelMag_r,modelMag_i,modelMag_z
0,0,601,1.182702,1.048251,0.626485,-0.220330,-0.618852,1.672996,1.233289,0.682468,-0.142070,-0.504366,0.734331,1.099892,0.676638,-0.033574,-0.338060,1.827431,1.014195,0.626034,-0.045304,-0.323146
1,1,788,0.779387,0.817574,0.363304,0.413028,0.380063,0.607737,0.842195,0.395966,0.461799,0.530203,0.531978,0.771724,0.338297,0.431661,0.502168,0.757070,0.792720,0.343134,0.387349,0.473419
2,2,427,0.067795,-0.272150,-0.465267,-0.522964,-0.494957,0.068706,-0.251791,-0.512839,-0.523744,-0.463003,0.137658,-0.298732,-0.536779,-0.513360,-0.441137,0.061153,-0.304420,-0.537547,-0.505100,-0.446280
3,3,864,-0.140096,0.151644,0.306794,0.482166,0.597045,-0.026932,0.215032,0.375341,0.535200,0.628807,-0.016343,0.274626,0.315620,0.512201,0.677785,0.024657,0.253596,0.335487,0.523894,0.642499
4,4,612,1.353323,1.112500,0.731795,0.107394,-0.146202,1.934783,1.140655,0.804959,0.183104,-0.043074,0.758448,1.155090,0.707018,0.206875,0.045031,1.863337,1.136884,0.686727,0.201007,0.059354


In [0]:
prediction=lgbm.predict_proba(test_robustscaled)

In [0]:
id_for_index = test_robustscaled['id']
id_for_index
result = pd.DataFrame(data=prediction, index=id_for_index, columns=lgbm.classes_)
result.head()

,GALAXY,QSO,REDDEN_STD,ROSAT_D,SERENDIPITY_BLUE,SERENDIPITY_DISTANT,SERENDIPITY_FIRST,SERENDIPITY_MANUAL,SERENDIPITY_RED,SKY,SPECTROPHOTO_STD,STAR_BHB,STAR_BROWN_DWARF,STAR_CARBON,STAR_CATY_VAR,STAR_PN,STAR_RED_DWARF,STAR_SUB_DWARF,STAR_WHITE_DWARF
id,,,,,,,,,,,,,,,,,,,
0,1.650877e-07,0.000690,1.047283e-09,0.003056,5.054955e-07,2.427812e-06,3.759432e-07,2.052227e-08,2.291070e-02,2.729645e-10,4.974978e-08,2.600437e-09,7.746653e-05,8.490804e-10,6.960480e-08,1.245172e-09,9.732613e-01,1.844626e-07,3.780031e-09
1,3.746388e-08,0.999995,8.968428e-11,0.000002,8.271462e-10,5.422563e-09,2.712653e-06,2.718255e-14,2.138487e-10,4.796450e-13,7.938892e-12,8.455560e-11,3.177984e-16,4.397999e-11,6.410802e-11,2.224389e-11,1.336041e-14,1.499933e-11,2.643832e-10
2,9.948994e-01,0.002276,1.633011e-08,0.002600,5.104456e-06,2.622276e-07,9.451983e-05,8.426369e-09,3.724001e-08,2.207501e-10,3.623382e-08,2.544106e-07,4.091553e-09,1.695325e-06,3.917475e-07,1.148372e-09,1.654719e-08,3.113396e-12,1.213302e-04
3,9.169495e-10,0.999995,1.802219e-10,0.000001,2.638248e-06,1.793747e-08,8.535286e-07,1.181426e-12,1.315661e-12,2.739862e-13,3.900435e-12,4.852755e-12,2.834364e-15,2.957237e-12,1.437569e-10,1.800666e-11,1.292724e-14,2.140598e-14,7.113488e-10
4,2.914235e-07,0.000056,1.815286e-08,0.000288,5.487637e-09,8.456546e-09,2.976138e-07,2.858253e-10,3.264299e-07,1.626863e-10,4.435771e-09,4.180654e-09,1.595758e-09,1.904234e-08,6.607099e-11,1.295465e-10,9.996552e-01,1.098281e-09,4.911107e-10


In [0]:
sample = pd.read_csv("/content/gdrive/My Drive/sample_submission.csv")
col_order = sample.columns[1:]
result = result[col_order]
result

,STAR_WHITE_DWARF,STAR_CATY_VAR,STAR_BROWN_DWARF,SERENDIPITY_RED,REDDEN_STD,STAR_BHB,GALAXY,SERENDIPITY_DISTANT,QSO,SKY,STAR_RED_DWARF,ROSAT_D,STAR_PN,SERENDIPITY_FIRST,STAR_CARBON,SPECTROPHOTO_STD,STAR_SUB_DWARF,SERENDIPITY_MANUAL,SERENDIPITY_BLUE
id,,,,,,,,,,,,,,,,,,,
0,3.780031e-09,6.960480e-08,7.746653e-05,2.291070e-02,1.047283e-09,2.600437e-09,1.650877e-07,2.427812e-06,0.000690,2.729645e-10,9.732613e-01,0.003056,1.245172e-09,3.759432e-07,8.490804e-10,4.974978e-08,1.844626e-07,2.052227e-08,5.054955e-07
1,2.643832e-10,6.410802e-11,3.177984e-16,2.138487e-10,8.968428e-11,8.455560e-11,3.746388e-08,5.422563e-09,0.999995,4.796450e-13,1.336041e-14,0.000002,2.224389e-11,2.712653e-06,4.397999e-11,7.938892e-12,1.499933e-11,2.718255e-14,8.271462e-10
2,1.213302e-04,3.917475e-07,4.091553e-09,3.724001e-08,1.633011e-08,2.544106e-07,9.948994e-01,2.622276e-07,0.002276,2.207501e-10,1.654719e-08,0.002600,1.148372e-09,9.451983e-05,1.695325e-06,3.623382e-08,3.113396e-12,8.426369e-09,5.104456e-06
3,7.113488e-10,1.437569e-10,2.834364e-15,1.315661e-12,1.802219e-10,4.852755e-12,9.169495e-10,1.793747e-08,0.999995,2.739862e-13,1.292724e-14,0.000001,1.800666e-11,8.535286e-07,2.957237e-12,3.900435e-12,2.140598e-14,1.181426e-12,2.638248e-06
4,4.911107e-10,6.607099e-11,1.595758e-09,3.264299e-07,1.815286e-08,4.180654e-09,2.914235e-07,8.456546e-09,0.000056,1.626863e-10,9.996552e-01,0.000288,1.295465e-10,2.976138e-07,1.904234e-08,4.435771e-09,1.098281e-09,2.858253e-10,5.487637e-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199986,1.521771e-04,2.474826e-05,1.476625e-09,2.475123e-08,7.617831e-06,8.858410e-05,7.964774e-05,8.968433e-05,0.587397,1.050109e-08,6.358533e-09,0.278466,1.101032e-08,1.324233e-01,1.507989e-08,5.067278e-07,1.214382e-13,4.297442e-09,1.269889e-03
199987,9.072986e-05,7.514539e-05,1.135927e-09,2.722545e-08,5.315492e-06,1.184515e-04,9.753321e-05,1.069020e-04,0.442613,5.597869e-09,1.208316e-08,0.396370,2.232228e-08,1.586654e-01,9.179842e-09,1.993136e-07,7.919551e-13,1.655619e-07,1.856639e-03
199988,6.281985e-05,1.410677e-05,1.974902e-09,1.122830e-08,3.510703e-05,7.371974e-05,1.677353e-04,6.591399e-05,0.500666,6.092023e-09,2.620039e-08,0.350283,8.474863e-09,1.472158e-01,5.124076e-09,1.758792e-07,8.193956e-13,9.311230e-08,1.415811e-03


In [0]:
result.to_csv('robustscaled_lgbm.csv',sep=',')

In [0]:
#train데이터로부터 log_loss대충 가늠해보기
from sklearn.model_selection import train_test_split
X_log = train_robustscaled.drop(columns=['type'], axis = 1)
y_log = train_robustscaled['type']
X_train, X_test, y_train, y_test = train_test_split(X_log, y_log, test_size= 0.2, random_state=42)

In [0]:
lgbm.fit(X_train,y_train)
lgbm.score(X_test,y_test)

0.8746468661716543

In [0]:
from sklearn.metrics import log_loss
y_pred_lgbm_prob = lgbm.predict_proba(X_test)
log_loss(y_true=y_test, y_pred=y_pred_lgbm_prob)

0.39106171334794815